In [222]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
df = pd.read_csv("fake_or_real_news.csv")
    


df.title = df.title.str.lower()





df= df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1)

df['title']=df['title'].str.lower()

y = df.label 


df.drop("label", axis=1)   



df.title = df.title.str.replace(r'http[\w:/\.]+','<URL>') 

df.title = df.title.str.replace(r'[^\.\w\s]','') 
df.title = df.title.str.replace(r'\.\.+','.') 

df.title = df.title.str.replace(r'\.',' . ') 

df.title = df.title.str.replace(r'\s\s+',' ') 

df.title = df.title.str.strip() 




In [223]:
df.title.values

array(['you can smell hillarys fear',
       'watch the exact moment paul ryan committed political suicide at a trump rally video',
       'kerry to go to paris in gesture of sympathy', ...,
       'antitrump protesters are tools of the oligarchy information',
       'in ethiopia obama seeks progress on peace security in east africa',
       'jeb bush is suddenly attacking trump . heres why that matters'], dtype=object)

In [224]:
from collections import Counter
all_text = ' '.join(df.title.values)
words = all_text.split()
u_words = Counter(words).most_common()
u_words_counter = u_words
u_words_frequent = [word[0] for word in u_words if word[1]>5] 

In [225]:

import matplotlib.pyplot as plt

import re

from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, LSTM, Embedding, Reshape
from keras.models import load_model, model_from_json

from sklearn.model_selection import train_test_split

import os
import urllib

from urllib.request import urlretrieve

from os import mkdir, makedirs, remove, listdir

from collections import Counter


with open('./dataglove/glove.6B.50d.txt','rb') as f:
    lines = f.readlines()
    
glove_weights = np.zeros((len(lines), 50))
words = []
for i, line in enumerate(lines):
    word_weights = line.split()
    words.append(word_weights[0])
    weight = word_weights[1:]
    glove_weights[i] = np.array([float(w) for w in weight])
word_vocab = [w.decode("utf-8") for w in words]

word2glove = dict(zip(word_vocab, glove_weights))
    

In [226]:
u_words_total = [k for k,v in u_words_counter]
word_vocab = dict(zip(word_vocab, range(len(word_vocab))))
word_in_glove = np.array([w in word_vocab for w in u_words_total])

words_in_glove = [w for w,is_true in zip(u_words_total,word_in_glove) if is_true]
words_not_in_glove = [w for w,is_true in zip(u_words_total,word_in_glove) if not is_true]

In [227]:
word2num = dict(zip(words_in_glove,range(len(words_in_glove))))
len_glove_words = len(word2num)
freq_words_not_glove = [w for w in words_not_in_glove if w in u_words_frequent]
b = dict(zip(freq_words_not_glove,range(len(word2num), len(word2num)+len(freq_words_not_glove))))
word2num = dict(**word2num, **b)
word2num['<Other>'] = len(word2num)
num2word = dict(zip(word2num.values(), word2num.keys()))
df_new = df[df['title'].notnull()]
int_text = [[word2num[word] if word in word2num else word2num['<Other>'] 
             for word in content.split()]  for content in df.title.values
           ]

In [228]:

num2word[len(word2num)] = '<PAD>'
word2num['<PAD>'] = len(word2num)

for i, t in enumerate(int_text):
    if len(t)<500:
        int_text[i] = [word2num['<PAD>']]*(500-len(t)) + t
    elif len(t)>500:
        int_text[i] = t[:500]
    else:
        continue

x = np.array(int_text)
y = (df.label.values=='REAL').astype('int')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [237]:
from scipy.stats import logistic
X_train=logistic.cdf(X_train)


In [245]:

x[1:2]


array([[9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824,
        9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824,
        9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824,
        9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824,
        9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824,
        9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824,
        9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824,
        9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824,
        9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824,
        9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824,
        9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824,
        9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824,
        9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824, 9824,
        9824, 9824, 9824, 9824, 9824, 

In [246]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format("../input/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin", binary=True)
import pandas as pd
import numpy as np
data=pd.read_csv("../input/train.csv")
textdata=data['question_text'].str.lower()
labels=data['target']
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_NB_WORDS=200000

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(textdata)
sequences = tokenizer.texts_to_sequences(textdata)

word_index = tokenizer.word_index

print('Found %s unique tokens.' % len(word_index))

x_traindata = pad_sequences(sequences, maxlen=200)

y_labels = to_categorical(np.asarray(labels))
model=word2vec
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
	embedding_vector = model.wv[model.wv.index2word[i]]
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding


array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1. 

In [229]:
model = Sequential()
model.add(Embedding(len(word2num), 50)) # , batch_size=batch_size
model.add(LSTM(64)
         )
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 50)          491250    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 520,755
Trainable params: 520,755
Non-trainable params: 0
_________________________________________________________________


In [238]:
batch_size = 50

model.fit(X_train, y_train, batch_size=batch_size, epochs=100, validation_data=(X_test, y_test))

Train on 5701 samples, validate on 634 samples
Epoch 1/5
5701/5701 [==============================] - 33s 6ms/step - loss: 0.7128 - acc: 0.5031 - val_loss: 0.4884 - val_acc: 0.7823
Epoch 2/5
5701/5701 [==============================] - 39s 7ms/step - loss: 0.6956 - acc: 0.4936 - val_loss: 0.5073 - val_acc: 0.7744
Epoch 3/5
5701/5701 [==============================] - 37s 7ms/step - loss: 0.6937 - acc: 0.5039 - val_loss: 0.5286 - val_acc: 0.7461
Epoch 4/5
5701/5701 [==============================] - 39s 7ms/step - loss: 0.6933 - acc: 0.4971 - val_loss: 0.5515 - val_acc: 0.7224
Epoch 5/5
5701/5701 [==============================] - 40s 7ms/step - loss: 0.6927 - acc: 0.5094 - val_loss: 0.5655 - val_acc: 0.7145


In [239]:
sentence = "8 classic football banners of our time".lower()
sentence_num = [word2num[w] if w in word2num else word2num['<Other>'] for w in sentence.split()]
sentence_num = [word2num['<PAD>']]*(500-len(sentence_num)) + sentence_num

sentence_num = np.array(sentence_num)
sentence_num=logistic.cdf(sentence_num)
model.predict(sentence_num[None,:])

array([[ 0.51062495]], dtype=float32)

In [240]:
model.evaluate(X_test, y_test)

634/634 [==============================] - 1s 2ms/step


[0.56549168197138455, 0.71451104194960002]